In [26]:
import psycopg2
conn=psycopg2.connect(user="postgres",
                      host="localhost",
                      password="root11",
                      port=5432,
                      database="eda")

print(conn) # checking if connection is success or not
cur=conn.cursor() # creating a cursor object.

<connection object at 0x00000241DA1AFAB0; dsn: 'user=postgres password=xxx dbname=eda host=localhost port=5432', closed: 0>


In [27]:
cur.execute("drop table if exists raw_data")
conn.commit()

In [28]:
cur.execute("""
create table raw_data(
department_name varchar(50),
sales varchar(50),
country varchar(50)
);


insert into raw_data values
('@#h*&#R&','nine 7','+91'),
('s@a#le&S','nine foUr','+91'),
('maNage*&#ment&','one 0 Zero','iNdia'),
('Da@@ta_te@m*','17','inDIA'),
('M*&^%$l','five five','+99'),
('W#eB*DeveL0Opm#en@!t','onE siX','ruSSIA');

""")
conn.commit()

In [29]:
cur.execute("SELECT table_name FROM information_schema.tables WHERE table_schema = 'public';")
if 'raw_data' in cur.fetchone():
    print("table created successfully")

table created successfully


In [30]:
'''cur.execute("""INSERT INTO raw_data values;""")
conn.commit()
# cur.execute("""update sort_animal set animal_in_field=animal_name;""")
conn.commit()'''

'cur.execute("""INSERT INTO raw_data values;""")\nconn.commit()\n# cur.execute("""update sort_animal set animal_in_field=animal_name;""")\nconn.commit()'

In [31]:
cur.execute("select * from raw_data")
for rows in cur.fetchall():
    print(rows)

('@#h*&#R&', 'nine 7', '+91')
('s@a#le&S', 'nine foUr', '+91')
('maNage*&#ment&', 'one 0 Zero', 'iNdia')
('Da@@ta_te@m*', '17', 'inDIA')
('M*&^%$l', 'five five', '+99')
('W#eB*DeveL0Opm#en@!t', 'onE siX', 'ruSSIA')


In [32]:
cur.execute("select department_name from raw_data;")
data=cur.fetchall()

In [61]:
"dkfjkddK".lower()

'dkfjkddk'

In [57]:
lst=['@#h*&#R&','s@a#le&S']
lst_new=[]

In [72]:
lst=['@#h*&#R&','s@a#le&S']
lst_new=[]

for data in lst:

    flag=''
    for ele in data:
        

        if ele.isalpha():
            flag=flag+ele
    lst_new.append(flag)

In [73]:
lst_new

['hr', 'sales']

In [ ]:
word_to_number = {'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}

input_string = 'nine'
output_number = word_to_number.get(input_string)

if output_number is not None:
    print(output_number)
else:
    print("Invalid input string")


In [81]:
'9'.isnumeric()

True

In [1]:
word_to_number = {'zero': 0, 'one': 1, 'two': 2, 'three': 3, 'four': 4, 'five': 5, 'six': 6, 'seven': 7, 'eight': 8, 'nine': 9}

a = 'one 0 zero'
result = ''

for word in a.split():
    if word.isnumeric():
        result += word
    elif word in word_to_number:
        result+= str(word_to_number.get(word))
        # print(result,type(result))

print(result)

100
